<a href="https://colab.research.google.com/github/geekysilento/Schizophrenia-Detection-CNN/blob/main/Schizophrenia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing and setting up the Dataset from Kaggle

In [1]:
import os
import time
import numpy as np
import pandas as pd

import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

import warnings
warnings.filterwarnings('ignore')

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

! for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done

a was unexpected at this time.


Reading CSV Data


In [2]:
demographic = pd.read_csv("button-tone-sz/demographic.csv")
demographic

,subject,group,gender,age,education
0,1,0,M,44,16.0
1,2,0,M,39,17.0
2,3,0,M,53,18.0
3,4,0,M,52,15.0
4,5,0,M,41,16.0
...,...,...,...,...,...
76,77,1,M,28,13.0
77,78,1,F,32,16.0
78,79,1,M,37,16.0
79,80,1,M,33,13.0


In [3]:
columns = pd.read_csv("button-tone-sz/time.csv")
columns

,sample,time_ms
0,1,-1500.0000
1,2,-1499.0234
2,3,-1498.0469
3,4,-1497.0703
4,5,-1496.0938
...,...,...
3067,3068,1495.1172
3068,3069,1496.0938
3069,3070,1497.0703
3070,3071,1498.0469


In [13]:
diagnosis_dict = dict(zip(demographic.subject, demographic[" group"]))
electrodes_list = list(pd.read_csv("button-tone-sz/columnLabels.csv").columns[4:])

print(diagnosis_dict)


print("Electrodes List \n",electrodes_list)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1}


In [5]:
def averaged_by_N_rows(a, n):
    """
    A function that averages n rows in a matrix
    """
    shape = a.shape
    assert len(shape) == 2
    assert shape[0] % n == 0
    b = a.reshape(shape[0] // n, n,
                  shape[1])
    mean_vec = b.mean(axis=1)
    return mean_vec

N_AVERAGED = 16
x_counter = 0
X = np.zeros((81 * 100,  9216 * len(electrodes_list) // N_AVERAGED), dtype="float32")
Y = np.zeros(len(X))

part1_path = "button-tone-sz"
part2_path = "button-tone-sz"


column_list = pd.read_csv("button-tone-sz/columnLabels.csv").columns
for person_number in tqdm(range(1, 81 + 1)):


    csv_path = f"{part1_path}/{person_number}.csv/{person_number}.csv"
    if not os.path.exists(csv_path):
        csv_path = f"{part2_path}/{person_number}.csv/{person_number}.csv"
    df = pd.read_csv(csv_path,
                header=None,
                names=column_list)
    trials_list = set(df.trial)


    for t1, trial_number in enumerate(trials_list):
        number_of_trials = len(df[df.trial == trial_number])
        if number_of_trials == 9216.0:
            current_sample_matrix = df[df.trial == trial_number][electrodes_list].values
            averaged_by_N = averaged_by_N_rows(current_sample_matrix, n=N_AVERAGED)
            averaged_by_N_big_vec = averaged_by_N.reshape(-1)
            X[x_counter] = averaged_by_N_big_vec.astype(np.float32)
            Y[x_counter] = diagnosis_dict[person_number]
            x_counter += 1
print("Total trials with the Appropriate number of measurements - ", x_counter)
X = X[: x_counter]
Y = Y[: x_counter]

print("Total trials with the Appropriate number of measurements- ", x_counter)

  0%|          | 0/81 [00:00<?, ?it/s]

Total trials with the Appropriate number of measurements -  7092
Total trials with the Appropriate number of measurements-  7092


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)
X_norm = (normalize(X.reshape(-1, 70), axis=0, norm='max')).reshape(X.shape)
X_train_norm, X_test_norm, Y_train_norm, Y_test_norm = train_test_split(X_norm, Y, test_size=0.2, shuffle=True, random_state=42)
_norm = X

X_train_reshaped = X_train_norm.reshape(X_train_norm.shape[0], -1, len(electrodes_list))
X_test_reshaped = X_test_norm.reshape(X_test_norm.shape[0], -1, len(electrodes_list))



In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define the model
model = Sequential()

# Add an LSTM layer with input shape adjustment
model.add(LSTM(128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), activation='tanh', return_sequences=True))
model.add(Dropout(0.3))

# Add another LSTM layer
model.add(LSTM(128, activation='tanh', return_sequences=True))
model.add(Dropout(0.3))

# Add a GRU layer
model.add(GRU(128, activation='tanh'))
model.add(Dropout(0.3))

# Add Dense layers with regularization
model.add(Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Dropout(0.4))

model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Dropout(0.4))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with a custom learning rate
custom_optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])

# Implement early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

Training the Model

In [8]:
# Train the model
history = model.fit(
    X_train_reshaped,
    Y_train_norm,
    batch_size=64,
    epochs=500,  # Adjust the number of epochs
    verbose=1,
    shuffle=True,
    validation_data=(X_test_reshaped, Y_test_norm),
    callbacks=[early_stopping]
)


Epoch 1/500
89/89 [==============================] - 23s 150ms/step - loss: 0.8341 - accuracy: 0.5942 - val_loss: 0.7342 - val_accuracy: 0.5927
Epoch 2/500
89/89 [==============================] - 12s 138ms/step - loss: 0.7112 - accuracy: 0.5967 - val_loss: 0.6969 - val_accuracy: 0.5927
Epoch 3/500
89/89 [==============================] - 12s 138ms/step - loss: 0.6928 - accuracy: 0.5983 - val_loss: 0.6878 - val_accuracy: 0.5927
Epoch 4/500
89/89 [==============================] - 12s 139ms/step - loss: 0.6846 - accuracy: 0.5983 - val_loss: 0.6819 - val_accuracy: 0.5927
Epoch 5/500
89/89 [==============================] - 12s 138ms/step - loss: 0.6801 - accuracy: 0.5983 - val_loss: 0.6794 - val_accuracy: 0.5927
Epoch 6/500
89/89 [==============================] - 12s 139ms/step - loss: 0.6772 - accuracy: 0.5983 - val_loss: 0.6779 - val_accuracy: 0.5927
Epoch 7/500
89/89 [==============================] - 12s 137ms/step - loss: 0.6759 - accuracy: 0.5983 - val_loss: 0.6779 - val_accuracy: